In [1]:
import openrouteservice as ors
import folium
from pandas import DataFrame
import pandas as pd
import math
import time

In [2]:
client = ors.Client(key='5b3ce3597851110001cf6248e87222039de944878943e770cf957d7d')

In [3]:
raw_dir = '../data/raw/'
cur_dir = '../data/curated/'

In [4]:
rent = pd.read_csv("../rent_data/renew(15)")
rent = rent[:5].reset_index()
school = pd.read_csv(f"{cur_dir}school_df_21_preprocessed.csv")

In [5]:
# type the coordinates u need
# coordinates = 
"""
function: draw route map, return route
param datas : map, route
"""
def mapping(coordinates, client):
    map = folium.Map(location=[37.50, 144.58], tiles='cartodbpositron', zoom_start=13)
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
        )
    folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]).add_to(map)
    return route, map


In [6]:

"""
function: calculate coordinates' distance
param datas : durations, distances
"""
def cal_distance(coordinates, client):
    matrix = client.distance_matrix(
    locations=coordinates,
    profile='driving-car',
    metrics=['distance', 'duration'],
    validate=False,
    )
    time.sleep(2.5)
    return matrix['durations'], matrix['distances']
# the unit of distance is m, the unit of durations is seconds


In [7]:

"""
function: The number of times the code needs to run to the one house and all train stations.
param datas : times_i(corresponding i which is in function), times_function
"""
#as api only apply 3500 times each time
def times(len):
    times_i = int(math.sqrt(3500))-1
    times = math.ceil(len/times_i)
    times_function = math.ceil((len+times)/times_i)
    return times_i, times_function



In [36]:
coordinates = []
distances = []
for j in range(len(school)):
    coordinates.append([school["X"][j], school["Y"][j]])

#print(coordinates)

print(len(school)/(times(len(school))[1]))

i=0
while i < int(len(school)/(times(len(school))[1])):
    coor_part = []
    for j in range(times(len(school))[1]):
        coor_part.append(coordinates[i+j])
        
    distance = cal_distance(coor_part, client)[1]
    distances.append(min(distance))
    
    i+=1
print(distances)
        
#print(cal_distance(coor_part, client)[0][0])


55.8780487804878


TypeError: 'list' object is not callable

In [34]:

for i in distances:
    i.pop(0)

index = distances.index(min(distances))
index

TypeError: 'list' object is not callable

In [21]:
train = pd.read_excel('../data/raw/train_station_19to20.xlsx')
coordinates = []

for j in range(len(train)):
            coordinates.append([train["Stop_long"][j], train["Stop_lat"][j]])
            
print(coordinates)

[[144.572217, -37.703362], [144.65063, -37.729258], [144.772307, -37.77776], [144.802193, -37.783063], [146.924518, -36.084263], [142.936912, -37.282202], [145.229517, -36.893644], [144.436857, -37.687517], [147.627615, -37.828717], [144.225443, -37.6043], [143.859452, -37.55879], [143.382233, -37.427875], [145.983914, -36.544549], [144.283012, -36.765673], [143.78362, -38.328807], [145.043009, -37.207202], [145.720762, -38.09911], [143.150932, -38.2289], [144.213801, -37.062836], [146.61138, -36.155634], [144.745368, -37.483497], [143.586658, -38.343378], [144.37977, -38.072835], [144.23112, -36.369235], [144.96955, -37.542571], [145.855943, -38.136453], [144.248382, -36.718528], [144.753463, -36.130985], [144.607753, -36.495017], [145.568086, -36.749135], [145.67422, -38.09103], [144.354992, -38.144241], [144.598718, -37.458825], [145.028202, -37.371709], [144.248993, -36.794838], [143.92443, -35.733123], [144.983565, -37.293208], [144.450602, -37.258277], [144.414402, -38.022428], [

In [37]:
"""
function: Find the nearest site to rent 
param datas : rent_address, stop_name, min_distance, min_duration
"""
def get_coordinates(rent, school):
    rent_address = []
    school_name = []
    min_distance = []
    min_duration = []
    for i in range(len(rent)):
        coordinates = []
        distances = []
        durations = []
        for j in range(len(school)):
            coordinates.append([school["X"][j], school["Y"][j]])
        for k in range(times(len(school))[1]):
            coor_part = []
            coor_part.append([rent["longitude"][i], rent["latitude"][i]])
            coor_part.extend(coordinates[k*times(len(coordinates))[0]:(k+1)*times(len(coordinates))[0]])
            #matrix = cal_distance(coor_part, client)
            distances = cal_distance(coor_part, client)[1]
            durations = cal_distance(coor_part, client)[0]
        index = distances.index(min(distances))
        rent_address.append(rent['listingSlug'][i])
        school_name.append(school['School_Name'][index])
        min_distance.append(distances[index])
        min_duration.append(durations[index])
    return rent_address, school_name, min_distance, min_duration


In [38]:

"""
function: put the required columns into a dataframe
param datas: dataframe
"""

def data_frame(rent, school):
    data = {'rent_address': get_coordinates(rent, school)[0], 'school_name': get_coordinates(rent, school)[1], 'min_distance': get_coordinates(rent, school)[2], 'min_duration': get_coordinates(rent, school)[3]}
    df = pd.DataFrame(data)
    return df

df = data_frame(rent, school)
df.to_csv(f"{cur_dir}/distances_durations_school21.csv", index=False)

/home/jnniu/miniconda3/envs/group55/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/jnniu/miniconda3/envs/group55/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/jnniu/miniconda3/envs/group55/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/jnniu/miniconda3/envs/group55/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_count

Timeout: 